In [2]:
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Activation, Dense, merge
from keras.layers.recurrent import LSTM

In [3]:
import numpy as np
from DatasetBuilder import DatasetBuilder

In [4]:
# Read data

FLDS = ['track_id', 'artist', 'title', 'similars', 'lyrics_url', 'sample_url']
DATASET = DatasetBuilder(True,
                         'D:\\PROJECT\\data\\train_data', 'D:\\PROJECT\\data\\train_lyrics',
                         'D:\\PROJECT\\data\\test_data', 'D:\\PROJECT\\data\\test_lyrics',
                         FLDS, 0.5)

DATASET.load_from_file('../data/db_test.neg', False, False)
DATASET.load_from_file('../data/db_test.pos', False, True)
DATASET.load_from_file('../data/db_train.neg', True, False)
DATASET.load_from_file('../data/db_train.pos', True, True)
DATASET.info()

#DATASET.read_data(False, 1000)
#DATASET.read_data(True, 10000)
#DATASET.dump('../data/db')
#DATASET.info()

# Join all text data into 2 lists to build vocabulary out of them
TRAIN_DATA_1 =  [data_item['data_1'] for data_item in DATASET.train_data['pos']] + \
                [data_item['data_1'] for data_item in DATASET.train_data['neg']]
TRAIN_DATA_2 =  [data_item['data_2'] for data_item in DATASET.train_data['pos']] + \
                [data_item['data_2'] for data_item in DATASET.train_data['neg']]
TRAIN_SCORES =  [data_item['score'] for data_item in DATASET.train_data['pos']] + \
                [data_item['score'] for data_item in DATASET.train_data['neg']]

TEST_DATA_1 =   [data_item['data_1'] for data_item in DATASET.test_data['pos']] + \
                [data_item['data_1'] for data_item in DATASET.test_data['neg']]
TEST_DATA_2 =   [data_item['data_2'] for data_item in DATASET.test_data['pos']] + \
                [data_item['data_2'] for data_item in DATASET.test_data['neg']]
TEST_SCORES =   [data_item['score'] for data_item in DATASET.test_data['pos']] + \
                [data_item['score'] for data_item in DATASET.test_data['neg']]

DataBuilder initialized.
Dataset Info
> Train set:
> > Positive: 10000
> > Negative: 10000
> Test set:
> > Positive: 1000
> > Negative: 1000


In [5]:
from keras.preprocessing import text
from nltk.stem import SnowballStemmer
import string

word_dictionary = {}
def unique_recursive_len(item):
    for it in item:
        for elem in it:
            if elem not in word_dictionary:
                word_dictionary[elem] = 1
            else:
                word_dictionary[elem] = word_dictionary[elem] + 1
    return len(word_dictionary)

def wordIsPrintable(word):
    printable = string.printable
    f = 1
    for c in word:
        if c not in printable:
            f = 0
            break
    return f

stemmer = SnowballStemmer("english")
TRAIN_DATA_1_ws = [[stemmer.stem(word) for word in text.text_to_word_sequence(t, lower=True, split=" ")] for t in TRAIN_DATA_1]
TRAIN_DATA_2_ws = [[stemmer.stem(word) for word in text.text_to_word_sequence(t, lower=True, split=" ")] for t in TRAIN_DATA_2]
TEST_DATA_1_ws = [[stemmer.stem(word) for word in text.text_to_word_sequence(t, lower=True, split=" ")] for t in TEST_DATA_1]
TEST_DATA_2_ws = [[stemmer.stem(word) for word in text.text_to_word_sequence(t, lower=True, split=" ")] for t in TEST_DATA_2]
number_of_words = unique_recursive_len(TRAIN_DATA_1_ws + TRAIN_DATA_2_ws + TEST_DATA_1_ws + TEST_DATA_2_ws)
print(number_of_words)

58403


In [6]:
stemmed_dict = {}

for word, count in word_dictionary.items():
    stemmed_word = stemmer.stem(word)
    if stemmed_word not in stemmed_dict:
        stemmed_dict[stemmed_word] = count
    else:
        stemmed_dict[stemmed_word] = stemmed_dict[stemmed_word] + count
print(len(word_dictionary), len(stemmed_dict))

58403 57786


In [7]:
VOCAB_SIZE = 25000

In [21]:
TRAIN_DATA_1_hot = [text.one_hot(t, VOCAB_SIZE, lower=True, split=" ") for t in TRAIN_DATA_1]
TRAIN_DATA_2_hot = [text.one_hot(t, VOCAB_SIZE, lower=True, split=" ") for t in TRAIN_DATA_2]
TEST_DATA_1_hot = [text.one_hot(t, VOCAB_SIZE, lower=True, split=" ") for t in TEST_DATA_1]
TEST_DATA_2_hot = [text.one_hot(t, VOCAB_SIZE, lower=True, split=" ") for t in TEST_DATA_2]

In [22]:
from keras.preprocessing import sequence

SEQUENCE_LENGTH = 200

TRAIN_DATA_1_hot = sequence.pad_sequences(TRAIN_DATA_1_hot, maxlen=SEQUENCE_LENGTH)
TRAIN_DATA_2_hot = sequence.pad_sequences(TRAIN_DATA_2_hot, maxlen=SEQUENCE_LENGTH)
TEST_DATA_1_hot = sequence.pad_sequences(TEST_DATA_1_hot, maxlen=SEQUENCE_LENGTH)
TEST_DATA_2_hot = sequence.pad_sequences(TEST_DATA_2_hot, maxlen=SEQUENCE_LENGTH)

In [16]:
# Have to change input shape (nb_samples, timesteps, input_dim)
TRAIN_DATA_1_hot = TRAIN_DATA_1_hot.reshape(len(TRAIN_DATA_1_hot), SEQUENCE_LENGTH, 1)
TRAIN_DATA_2_hot = TRAIN_DATA_2_hot.reshape(len(TRAIN_DATA_2_hot), SEQUENCE_LENGTH, 1)
TEST_DATA_1_hot = TEST_DATA_1_hot.reshape(len(TEST_DATA_1_hot), SEQUENCE_LENGTH, 1)
TEST_DATA_2_hot = TEST_DATA_2_hot.reshape(len(TEST_DATA_2_hot), SEQUENCE_LENGTH, 1)

In [23]:
EMBED_VEC_LEN = 32
output_dim = 20

input_data_1 = Input(shape=(SEQUENCE_LENGTH,))
input_data_2 = Input(shape=(SEQUENCE_LENGTH,))

embed = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBED_VEC_LEN, input_length=SEQUENCE_LENGTH)

embed_1 = embed(input_data_1)
embed_2 = embed(input_data_2)

lstm = LSTM(output_dim, input_shape=(EMBED_VEC_LEN, 1))

encoded_1 = lstm(embed_1)
encoded_2 = lstm(embed_2)

merged_vector = merge([encoded_1, encoded_2], mode='concat', concat_axis=-1)

predictions = Dense(1, activation='sigmoid')(merged_vector)

model = Model(input=[input_data_1, input_data_2], output=predictions)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 200, 32)       800000      input_5[0][0]                    
                                                                   input_6[0][0]                    
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 20)            4240        embedding_3[0][0]       

In [24]:
from random import shuffle

# shuffle data a bit
index_shuf = list(range(len(TRAIN_SCORES)))
shuffle(index_shuf)

TRAIN_DATA_1_hot_shuf = np.asarray([TRAIN_DATA_1_hot[i] for i in index_shuf]).reshape(TRAIN_DATA_1_hot.shape)
TRAIN_DATA_2_hot_shuf = np.asarray([TRAIN_DATA_2_hot[i] for i in index_shuf]).reshape(TRAIN_DATA_2_hot.shape)
TRAIN_SCORES_shuf = np.asarray([TRAIN_SCORES[i] for i in index_shuf]).reshape(np.asarray(TRAIN_SCORES).shape)

In [25]:
# Divide train dataset into test and train parts
# !!! I don't know why but with actual test dataset ANN gives bad results on evaluation
THRESHOLD = 2000
TEST_D1 = TRAIN_DATA_1_hot_shuf[:THRESHOLD]
TEST_D2 = TRAIN_DATA_2_hot_shuf[:THRESHOLD]
TEST_S = TRAIN_SCORES_shuf[:THRESHOLD]

In [26]:
length = len(TRAIN_SCORES)
model.fit([TRAIN_DATA_1_hot_shuf[THRESHOLD:length], TRAIN_DATA_2_hot_shuf[THRESHOLD:length]], TRAIN_SCORES_shuf[THRESHOLD:length], nb_epoch=10, batch_size=16)

Epoch 1/10
18000/18000 [==============================] - 74s - loss: 0.4372 - acc: 0.7878    
Epoch 2/10
18000/18000 [==============================] - 76s - loss: 0.1626 - acc: 0.9452    
Epoch 3/10
18000/18000 [==============================] - 77s - loss: 0.1132 - acc: 0.9642    
Epoch 4/10
18000/18000 [==============================] - 77s - loss: 0.0967 - acc: 0.9706    
Epoch 5/10
18000/18000 [==============================] - 77s - loss: 0.0823 - acc: 0.9749    
Epoch 6/10
18000/18000 [==============================] - 77s - loss: 0.0741 - acc: 0.9776    
Epoch 7/10
18000/18000 [==============================] - 78s - loss: 0.0642 - acc: 0.9789    
Epoch 8/10
18000/18000 [==============================] - 78s - loss: 0.0610 - acc: 0.9812    
Epoch 9/10
18000/18000 [==============================] - 78s - loss: 0.0557 - acc: 0.9819    
Epoch 10/10
18000/18000 [==============================] - 79s - loss: 0.0508 - acc: 0.9833    


In [27]:
scores = model.evaluate([TEST_D1, TEST_D2], TEST_S, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2000/2000 [==============================] - 1s     
Accuracy: 97.10%


In [28]:
def save_model(model, name, folder=''):
    model_json = model.to_json()
    with open(folder + name + '.json', 'w') as json_file:
        json_file.write(model_json)
    model.save_weights(folder + name + '.h5')
    return {
        'model_file': folder + name + '.json',
        'weights_file': folder + name + '.h5'
    }

In [73]:
from keras.models import model_from_json
def load_model(model_file, weights_file):
    with open(model_file, 'r') as json_file:
        loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(weights_file)
    return loaded_model

In [29]:
save_model(model, '../trained_models/model_final')

{'model_file': '../trained_models/model_final.json',
 'weights_file': '../trained_models/model_final.h5'}